# Face Detection using SSD and the Caffe pre-trained model

*by Georgios K. Ouzounis, June 21st, 2021*

This notebook demonstrates face detection in still images using the SSD detector configured with the Caffe pretrained model 

## Copy the model files

We need the configuration file and the pre-trained weights

In [ ]:
%mkdir model/
%cd model

In [ ]:
!wget https://raw.githubusercontent.com/georgiosouzounis/face-detection-ssd-caffe/main/model/deploy.prototxt.txt

In [ ]:
!wget https://github.com/georgiosouzounis/face-detection-ssd-caffe/raw/main/model/res10_300x300_ssd_iter_140000.caffemodel

In [ ]:
%cd ../

## Import the libraries

In [ ]:
# import the relevant libraries
import numpy as np
import cv2 # openCV

In [ ]:
# check the opencv version
if cv2.__version__ < '4.5.2':
  print("opencv version: ", cv2.__version__)
  print("please upgrade your opencv installation to the latest")

In [ ]:
# if the openCV version is < 4.4.0 update to the latest otherwise skip this step
!pip install opencv-python==4.5.2.52

## Read the model and initialize the detector

In [ ]:
# load the serialized model from the local copy in model/
model_cfg = "model/deploy.prototxt.txt"
model_weights = "model/res10_300x300_ssd_iter_140000.caffemodel"

In [ ]:
# read the model
detector = cv2.dnn.readNetFromCaffe(model_cfg, model_weights)

## Get a test image

Set the path to an image containing a face in your own Google Drive or use the example as shown:

In [ ]:
%cp /content/drive/MyDrive/macron.jpg .

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
test_img = "macron.jpg"

In [ ]:
# load the test image and create an image blob
image = cv2.imread(test_img)
(h, w) = image.shape[:2]

In [ ]:
# display the image 
from google.colab.patches import cv2_imshow
cv2_imshow(image)

## Deploy the detector

In [ ]:
# set the intensity scaling factor; 1 in this case, i.e. original image intensities
scalefactor = 1.0
# set the new dimensions for image resizing to match the network requirements
new_size = (300, 300)

# create a blob using OpenCV's DNN functionality and by performing mean subtraction 
# to normalize the input
blob = cv2.dnn.blobFromImage(image, scalefactor, new_size, (127.5, 127.5, 127.5), swapRB=True, crop=False)

In [ ]:
# set the blob as input to the network
detector.setInput(blob)
# compute the forward pass - detect faces if any
detections = detector.forward()
detections.shape

## Analyze the results

Let us review the detections. The shape of the detections is expected to be in the following format: ```[1, 1, N, 7]```, where N is the number of detected bounding boxes. For each detection, the description has the format: ```[image_id, label, conf, x_min, y_min, x_max, y_max]```.

In [ ]:
detections[0][0][0]

In [ ]:
len(detections[0][0])

In [ ]:
# set the confidence threshold
confidence_threshold = 0.5

In [ ]:
# loop over the detections
for i in range(0, detections.shape[2]):
  # extract the confidence (i.e., probability) associated with the prediction
  confidence = detections[0, 0, i, 2]
  # ignore weak detections
  if confidence > confidence_threshold:
    # compute the (x, y)-coordinates of the bounding box for the detected object
    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")
    # draw the bounding box of the detected face
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
    # print the probability of this detection
    text = "confidence: {:.2f}%".format(confidence * 100)
    y = startY - 10 if startY - 10 > 10 else startY + 10
    cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

In [ ]:
# show the output image
cv2_imshow(image)